# Object modification with ORM

The world changes over time, and so does your data. When you have a new object to store, you create it in a database for further use; if you don't need it anymore, you delete it. When the object's properties change, you make an update. These three actions along with reading are known as CRUD (Create, Read, Update, Delete) operations.

# Create

Learning is hard, so we decide to take a break and make a truly entertaining computer game for ourselves. The galaxy is in danger, and our brave space team must prevent the alien invasion. The main weapons of our team are diplomacy and science. This time, we need only two models to start the game:

In [ ]:
from django.db import models


class Alien(models.Model):
    db_column = models.CharField(max_length=32)
    distance_to_galaxy = models.IntegerField()
    threat = models.IntegerField()
    speed = models.IntegerField()


class Weapon(models.Model):
    db_column = models.CharField(max_length=32)
    quantity = models.IntegerField()
    power = models.IntegerField()
    coverage_distance = models.IntegerField()

First, we should create some weapons for our heroes. Let's equip them with eloquence to convince aliens to go away and jammers to modify the scouting signals:

In [ ]:
eloquence = Weapon.objects.create(
   type='eloquence', power=100, coverage_distance=100, quantity=10
)
jammers = Weapon(type='jammer', power=10, coverage_distance=1000, quantity=50)
jammers.save()

Two methods illustrate how you can create new objects in a database. We create eloquence with the help of Object Manager. It has the method create where you can pass all the parameters your object has. If the result of this operation is successful, by calling it, you save the object to the database and return the instance of the Weapon class.

The second method is to create an instance of a class and then save it manually. The two methods are pretty much the same, so you can use whichever you like most.

# Delete

The alien invasion is coming. Meanwhile, we hope that our game will attract gamers and won't be boring. For each session, we'll create hundreds or even thousands of aliens to make the battle hard. If we don't clear the database of all the defeated aliens, operations will become slower, and we will eventually run out of disk space.

The first enemy comes from a nearby galaxy, located only 23 solar years from ours:

In [ ]:
et_alien = Alien.objects.create(type='ET', distance_to_galaxy=23, threat=70, speed=5)


In five moves, it crosses the border, so the player can apply eloquence to deceive the opponent and make it back up. The power of eloquence is 100 and the threat is 70, so in one move the player can resist the first invasion!

We do not need the et_alien anymore, so let's delete it:

In [ ]:
et_alien.delete()

# Update
We have studied two main powers of modification: create and delete. The third power is to change an existing object. When the properties of an object change, we should update them in the database.

The next enemy of the galaxy is the Predator, an opponent that can hardly be defeated in a single move since it comes from the deep space, and our weapons are not strong enough.

In [ ]:
predator = Alien.objects.create(type='Predator', distance_to_galaxy=550, threat=40, speed=30)

Our jammers create obstacles for signals in space, so the enemy loses direction. The player applies this weapon on the next move. The number of jammers decreased by one, and simultaneously the threat is diminished:

In [ ]:
jammers.quantity -= 1
jammers.save()

predator.distance_to_galaxy -= predator.speed
predator.threat -= jammers.power
predator.save()

Updating an object is a two-step operation. We change the attributes of an object and call the save method as for manual creation of an object.

In case of error, we used change only in one place. To avoid this, the best practice is to use atomic as a decorator:

In [ ]:
from django.db import IntegrityError, transaction

@transaction.atomic
def changes():
    jammers.quantity -= 1
    jammers.save()

    predator.distance_to_galaxy -= predator.speed
    predator.threat -= jammers.power
    predator.save()

# Modification of QuerySet

We can modify each object as we like, but remember — aliens may come in a swarm. Can we apply our weapons to all of them simultaneously? With the approach we've just considered, the answer would be no. If only we could use a QuerySet for this task… The good news is we surely can!

Space bugs come in a pack of three:



In [ ]:
Alien.objects.bulk_create([
    Alien(type='Space Bug', distance_to_galaxy=30, threat=150, speed=12) for _ in range(3)
])

In [ ]:
eloquence.quantity -= 1
eloquence.save()

space_bug = Alien.objects.filter(type='Space Bug').first()

Alien.objects.filter(type='Space Bug').update(
   distance_to_galaxy=space_bug.distance_to_galaxy - space_bug.speed,
   threat=space_bug.threat - eloquence.power
)

The bugs are not defeated yet, but we update their position and threat in one call, not three. We get an Object Manager of a model, filter out objects that we want, and call update on the QuerySet. The syntax is the same: we pass parameters and their new values to the method, and Django does the rest of the work with the database.

Finally, the player applies eloquence again and the bugs are convinced to stop their invasion. Again, we call the method on a QuerySet:

In [ ]:
Alien.objects.filter(type='Space Bug').delete()


# Other methods in Django ORM
Now that the galaxy is saved, we can return to our planet and discuss other ORM methods, this time in a more peaceful way. In addition to the well-known methods such as ```filter(), get(), update(), all(), and delete()```, Django ORM provides a lot of other methods that allow us to exploit the power of SQL queries. 

In [ ]:
class Movie(models.Model):
    title = models.CharField(max_length=100)
    genre = models.CharField(max_length=20)
    release_year = models.IntegerField()
    director = models.ForeignKey(Director, on_delete=models.CASCADE)

    def __str__(self):
        return self.title

As you may well have noticed, the movie table has a foreign key relationship with the director table, so let us define the Director model as well:

In [ ]:
class Director(models.Model):
    name = models.CharField(max_length=100)
    email = models.EmailField(null=True, blank=True)
    address = models.TextField()
    
    def __str__(self):
        return self.name

At first, we would like to mention the values() and values_list() methods. These methods are quite similar. Both return Python objects instead of QuerySet objects. The major difference between them is that the first one will return dictionaries while the second one returns tuples:

In [ ]:
movies_dict = Movie.objects.values()
print(movies_dict)

# <QuerySet [{'id': 1, 'title': 'Bonnie and Clyde', 'genre': 'action', 'release_year': 1967, 'director': 'Arthur Penn'}, {'id': 2, 'title': 'The Seven Samurai', 'genre': 'action', 'release_year': 1954, 'director': 'Akira Kurosawa'}, {'id': 3, 'title': 'Doctor Zhivago', 'genre': 'drama', 'release_year': 1965, 'director': 'David Lean'}, {'id': 4, 'title': 'Rocky', 'genre': 'adventure', 'release_year': 1976, 'director': 'John G. Avildsen'}, {'id': 5, 'title': 'Braveheart', 'genre': 'action', 'release_year': 1995, 'director': 'Mel Gibson'}]>


movies_tuple = Movie.objects.values_list()
print(movies_tuple)

# <QuerySet [(1, 'Bonnie and Clyde', 'action', 1967, 'Arthur Penn'), (2, 'The Seven Samurai', 'action', 1954, 'Akira Kurosawa'), (3, 'Doctor Zhivago', 'drama', 1965, 'David Lean'), (4, 'Rocky', 'adventure', 1976, 'John G. Avildsen'), (5, 'Braveheart', 'action', 1995, 'Mel Gibson')]>

The order_by() method changes the default ordering of the QuerySet. By default, the order is based on the primary key (in our case, the id field). If we want our QuerySet to be ordered by release_year, the order_by method is exactly what we need:

In [ ]:
ordered_movies = Movie.objects.order_by('release_year')
print(ordered_movies)

# <QuerySet [<Movie: The Seven Samurai>, <Movie: Doctor Zhivago>, <Movie: Bonnie and Clyde>, <Movie: Rocky>, <Movie: Braveheart>]>

You also can sort the data in descending order by adding a minus (-) before the value:



In [ ]:
descend_ordered_movies = Movie.objects.order_by('-release_year')
print(descend_ordered_movies)

# <QuerySet [<Movie: Braveheart>, <Movie: Rocky>, <Movie: Bonnie and Clyde>, <Movie: Doctor Zhivago>, <Movie: The Seven Samurai>]>

Lastly, we want to mention the select_related() method that can be used to improve database performance by retrieving all related data without performing multiple database searches. As you remember, our movie table has a foreign key relationship to the director table. To retrieve all related data, including the director at the first database lookup, we can use select_related() like so:

In [ ]:
rocky = Movie.objects.select_related('director').filter(title='rocky')
print(rocky.director) # director has already been retrieved. Database is not hit again.


# <Director: John G. Avildsen>

If you want to discover more about the methods we presented and some other interesting Django ORM methods like latest(), earliest() or in_bulk(), head over to the Django's official documentation page.

# PRACTICE


### Create Object in the Database
You have a model Task with fields description, is_done, priority:
```
class Task(models.Model):
    description = models.CharField(max_length=256)
    is_done = models.BooleanField()
    priority = models.IntegerField()
```

Create a new Task with description "walk to the grocery"(without quotes) and priority 5. The task is not done yet.

Save it to the database.

In [ ]:
Task.objects.create(description='walk to the grocery',priority=5,is_done=False)

### 2) Update Object in the Database
You have a model Task with fields description, is_done, priority:
```
class Task(models.Model):
    description = models.CharField(max_length=256)
    is_done = models.BooleanField()
    priority = models.IntegerField()
```
Update priority from 100 to 100500 for the task with description "call mom"(without quotes)

In [ ]:
# Fetch the specific task object
task_to_update = Task.objects.get(description="call mom")

# Modify its priority attribute
task_to_update.priority = 100500

# Save the changes to the database
task_to_update.save()

### 3) ORM puzzle
We created the Student model in order to populate our database with data containing information about all our students. Somehow the lines got all mixed up. Arrange the lines of code below, in a way that they form the valid Student model. The model fields must be in alphabetical order!

In [ ]:
from django.db import models

class Student(models.Model):
    address = models.TextField()
    name = models.CharField(max_length=100)
    school = models.CharField(max_length=100)

all_students = Student.objects.ordered_by('school')

# Excluding data

The Quidditch league is gaining popularity thanks to its new site we are currently developing. Users are keen on knowing the result of each game and are eager to explore interesting statistics about previous tournaments. We are working with two models: Team and Game. Our database, impressively large, contains all games from 1674 to the present day, thanks to the archives from the Hogwarts library. Given the size and scope of this data, we prefer to keep it in the database to enable us to access all the necessary information quickly:

```python
from django.db import models


class Team(models.Model):
    name = models.CharField(max_length=64)


class Game(models.Model):
    home_team = models.ForeignKey(Team, related_name='game_at_home', on_delete=models.CASCADE)
    home_team_points = models.IntegerField()
    rival_team = models.ForeignKey(Team, related_name='rival_game', on_delete=models.CASCADE)
    rival_team_points = models.IntegerField()
    date = models.DateField()

```

```pyhton
from datetime import date

falmouth_falcons = Team.objects.create(name="Falmouth Falcons")
montrose_magpies = Team.objects.create(name="Montrose Magpies")
tutshill_tornados = Team.objects.create(name="Tutshill Tornados")
appleby_arrows = Team.objects.create(name="Appleby Arrows") 

Game.objects.create(home_team=falmouth_falcons, home_team_points=15,
                    rival_team=montrose_magpies, rival_team_points=12, date=date(1674, 5, 6))

Game.objects.create(home_team=falmouth_falcons, home_team_points=34,
                    rival_team=tutshill_tornados, rival_team_points=8, date=date(1774, 9, 30))

Game.objects.create(home_team=appleby_arrows, home_team_points=10,
                    rival_team=montrose_magpies, rival_team_points=19, date=date(1779, 7, 15))

Game.objects.create(home_team=tutshill_tornados, home_team_points=7,
                    rival_team=appleby_arrows, rival_team_points=27, date=date(2018, 6, 25))

Game.objects.create(home_team=montrose_magpies, home_team_points=24,
                    rival_team=tutshill_tornados, rival_team_points=16, date=date(1907, 5, 12))
```



If you're familiar with the filter method of Django's Object Manager, you'll easily follow what the exclude method does. When filtering out data, we look for objects that satisfy certain conditions. Conversely, the exclude method allows us to remove objects from a QuerySet when a specific condition is met. Compare two queries:

In [ ]:
# All home games for Falmouth Falcons
Game.objects.filter(home_team=falmouth_falcons)

# All games excluding games where Falmouth Falcons was a home team
Game.objects.exclude(home_team=falmouth_falcons)

# Collections for lookups
special_home_games = Game.objects.exclude(home_team=tutshill_tornados) \
                                 .exclude(home_team=appleby_arrows)



But what if we have multiple conditions? Should we resort to extensive copy-pasting, or is there a smarter move? We recommend the latter. Let's use a Python list to filter what we need:

In [ ]:
special_home_games = Game.objects.filter(home_team__in=[falmouth_falcons, montrose_magpies])